In [2]:
import nltk
from nltk.corpus import brown
import numpy as np

In [3]:
# Let us add the project package into the model, so we can share 
# code with iPython.
import sys 
project_package = '/home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/'
if project_package not in sys.path:
    sys.path.append(project_package)
from shared import utils
from grasshopper import model as GH

In [41]:
reload(GH)
reload(utils)

<module 'shared.utils' from '/home/luis/Dropbox/OnlineDocuments/HarvardSchoolWork/Fall2015/cs182/project/shared/utils.pyc'>

In [9]:
original = brown.sents('ca19')
cleanDoc, mapping = utils.cleanDocument(original)
W = GH.docToMatrix(original, sim_fun=utils.cosineSim)
WClean = GH.docToMatrix(cleanDoc, sim_fun=utils.cosineSim)

In [10]:
# According to the paper, alpha = 0.25 and lambda = 0.5 where the 
# best parameters
lamb = 0.5
alpha = 0.25
r = utils.decayDistribution(alpha, len(W))
lsum = 5
results = GH.grasshopper(W, r, lamb, lsum)

In [11]:
rClean = utils.decayDistribution(alpha, len(WClean))
resultsClean = GH.grasshopper(WClean, rClean, lamb, lsum)

In [13]:
print results
print resultsClean

[(0, (0.015497071508764865+0j)), (102, 0.80360672696601831), (112, 0.79798967636799589), (101, 0.79652141447876801), (70, 0.79536920454736371)]
[(1, (0.017308867439736603-0j)), (37, 0.83458070810922114), (50, 0.81806194866979443), (48, 0.80492121346789014), (9, 0.80480230529447583)]


In [14]:
for (i, p) in sorted(resultsClean, key=lambda x: x[0]):
    print ' '.join(brown.sents('ca19')[i])

The current monthly payroll comes to about $15,000,000 .
Sufficient lay-offs of union employees to bring about a 10 per cent cut in the union payroll expense .
For a number of years the board used a machine to keep a permanent record but abandoned the practice about two years ago .
The vice president of the City Council complained yesterday that there are `` deficiencies '' in the city's snow clearing program which should be corrected as soon as possible .
Werner criticized


In [15]:
for (i, p) in sorted(results, key=lambda x: x[0]):
    print ' '.join(brown.sents('ca19')[i])

The Baltimore and Ohio Railroad announced yesterday it would reduce the total amount of its payroll by 10 per cent through salary cuts and lay-offs effective at 12:01 A.M. next Saturday .
Soon after 10 A.M. , when police reached the 1-1/2-story brick home in the Franklin Manor section , 15 miles south of here on the bay , in response to a call from the Dresbach's other son , Lee , 14 , they found Mrs. Dresbach's body on the first-floor bedroom floor .
While availability of mortgage money has been a factor in encouraging apartment construction , the generally high level of prosperity in the past few years plus rising consumer income are among the factors that have encouraged builders to concentrate in the apartment-building field .
Although economic and personal circumstances vary widely among those now choosing apartments , Leo J. Pantas , vice president of a hardware manufacturing company , pointed out recently that many apartment seekers seem to have one characteristic in common : a 

In [19]:
W1 = GH.docToMatrix(cleanDoc, vec_fun=utils.logFrequency, sim_fun=utils.threshHoldCosineSim)
W2 = GH.docToMatrix(cleanDoc, vec_fun=utils.frequency, sim_fun=utils.threshHoldCosineSim)

In [21]:
W2

array([[ 0.09522161,  0.02631579,  0.        , ...,  0.        ,
         0.        ,  0.11461988],
       [ 0.02631579,  0.34375   ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.24314703, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.2627551 ,
         0.        ,  0.02857143],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.15625   ,  0.        ],
       [ 0.11461988,  0.        ,  0.        , ...,  0.02857143,
         0.        ,  0.21975309]])

In [23]:
results1 = GH.grasshopper(W1, rClean, lamb, lsum)
results2 = GH.grasshopper(W2, rClean, lamb, lsum)

In [24]:
results1, results2

([(1, (0.017127102147263807-0j)),
  (26, 0.72523876747911331),
  (32, 0.71124544009738178),
  (37, 0.71265329939339295),
  (25, 0.70756935891690442)],
 [(1, (0.017317129974008406+0j)),
  (37, 0.83573162991964156),
  (50, 0.82023080917626234),
  (48, 0.8065918172806692),
  (9, 0.80647256755748287)])

In [25]:
for (i, p) in sorted(results1, key=lambda x: x[0]):
    print ' '.join(brown.sents('ca19')[i])

The current monthly payroll comes to about $15,000,000 .
His condition was reported to be fair .
Police said the children's mother , Mrs. Eleanor Somerville , was visiting next door when the fire occurred .
Cites discrepancies
For a number of years the board used a machine to keep a permanent record but abandoned the practice about two years ago .


In [26]:
for (i, p) in sorted(results2, key=lambda x: x[0]):
    print ' '.join(brown.sents('ca19')[i])

The current monthly payroll comes to about $15,000,000 .
Sufficient lay-offs of union employees to bring about a 10 per cent cut in the union payroll expense .
For a number of years the board used a machine to keep a permanent record but abandoned the practice about two years ago .
The vice president of the City Council complained yesterday that there are `` deficiencies '' in the city's snow clearing program which should be corrected as soon as possible .
Werner criticized


In [2]:
def createSummaries(sum_algo, abs_path, out_path, multiDocument = False, k = 5):
    # Extracted from the input folder name 
    setID = abs_path.split("/")[-1]
    
    # Stores DocumentIDs
    docIDs = []
    
    # Create document set that we wish to evaluate
    D = []
    for filename in os.listdir(abs_path):
        # Only take files that we've parsed before!
        tmp = filename.split('.')
        if tmp[0] == 'parsed':
            docIDs.append(tmp[1])
            filepath = os.path.join(abs_path, filename)
            with open(filepath) as inputDoc:
                D.append(inputDoc.readlines())
            
    # Pass this to the algorithm which should return the summary as
    # a list of sentences.
    if multiDocument:
        summary = sum_algo(D, k, multiDocument)
        # Write out the summary
        filepath = os.path.join(out_path, "Summary.{}.txt".format(setID))
        print filepath
        with open(filepath, 'w') as out:
            for s in summary:
                out.write("{}.\n".format(s.strip()))
    else:
        for i in range(len(D)):
            summary = sum_algo([D[i]], k)
            filepath = os.path.join(out_path, "Summary.{}.txt".format(docIDs[i]))
            print filepath
            with open(filepath, 'w') as out:
                for s in summary:
                    out.write("{}.\n".format(s.strip()))

In [3]:
# Just doing it for multiple documents
def grassHopper(D, k, multiDocument):
    # Concatenate into one big document if possible!
    original = [s for d in D for s in d]
    cleanDoc, mapping = utils.cleanDocument(original)
    WClean = GH.docToMatrix(cleanDoc, sim_fun=utils.threshHoldCosineSim)
    
    lamb = 0.5
    alpha = 0.25
    r = utils.decayDistribution(alpha, len(WClean))
    resultsClean = GH.grasshopper(WClean, r, lamb, k)
    
    summary = []
    for (i, p) in sorted(resultsClean, key=lambda x: x[0]):
        summary.append(original[i])
        
    return summary